In [5]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, Dropout, LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

In [7]:
data = pd.read_csv('data/petr4_treinamento.csv')
data = data.dropna()
train_data = data.iloc[:, 1:2].values
max_val_data = data.iloc[:, 2:3].values

In [8]:
scale = MinMaxScaler(feature_range=(0,1))
train_data_scaled = scale.fit_transform(train_data)
max_val_data_scaled = scale.fit_transform(max_val_data)

In [9]:
X = []
y1 = []
y2 = []

for i in range(90, 1242):
    X.append(train_data_scaled[i-90:i, 0])
    y1.append(train_data_scaled[i, 0])
    y2.append(max_val_data_scaled[i, 0])
X, y1, y2 = np.array(X), np.array(y1), np.array(y2)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

In [10]:
y = np.column_stack((y1, y2))

In [11]:
model = Sequential()
model.add(InputLayer(input_shape=(X.shape[1], 1)))
model.add(LSTM(units=100, return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(Dropout(0.3))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.3))

model.add(LSTM(units=50))
model.add(Dropout(0.3))

model.add(Dense(units=1, activation='linear'))

c:\Users\wilso\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
c:\Users\wilso\AppData\Local\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [12]:
model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mean_absolute_error'])

In [18]:
model.fit(X, y, epochs=100, batch_size=32)

Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 14s 149ms/step - loss: 0.0688 - mean_absolute_error: 0.1886
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 12s 201ms/step - loss: 0.0170 - mean_absolute_error: 0.1024
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 8s 123ms/step - loss: 0.0178 - mean_absolute_error: 0.1026
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 130ms/step - loss: 0.0111 - mean_absolute_error: 0.0812
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 138ms/step - loss: 0.0107 - mean_absolute_error: 0.0781
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 134ms/step - loss: 0.0105 - mean_absolute_error: 0.0785
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 136ms/step - loss: 0.0085 - mean_absolute_error: 0.0708
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 129ms/step - loss: 0.0076 - mean_absolute_error: 0.0674
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 122ms/step - loss: 0.0086 - mean_absolute_error: 0.0701
Epoch 10/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 111ms/step - loss: 0.0084 - mean_absolute_error: 0.0698
Epoch 11/100
36/3

In [13]:
test_data = pd.read_csv('data/petr4_teste.csv')
y_open = test_data.iloc[:, 1:2].values
y_high = test_data.iloc[:, 2:3].values

In [14]:
complete_data = pd.concat((data['Open'], test_data['Open']), axis=0)
inputs = complete_data[len(complete_data) - len(test_data) - 90:].values
inputs = inputs.reshape(-1,1)
inputs = scale.transform(inputs)

In [15]:
X_test = []
for i in range(90, 112):
    X_test.append(inputs[i-90:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [16]:
predicted_open = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


In [ ]:
print(mean_absolute_error(y_open.ravel(), predicted_open[:, 0]))
print(mean_absolute_error(y_high.ravel(), predicted_open[:, 1]))

In [ ]:
plt.plot(y_open, color='red', label='Preço abertura real')
plt.plot(y_high, color='black', label='Preço alta real')

plt.plot(predicted_open[:, 0], color='blue', label='Previsões abertura')
plt.plot(predicted_open[:, 1], color='orange', label='Previsões alta')

plt.title('Previsão preço das ações')
plt.xlabel('Tempo')
plt.ylabel('Valor')
plt.legend()